## R-ArcGIS Bridge and R Notebooks

In this notebook, working with arcgisbinding in a notebook environment will be demonstrated. The workflow will be as follows:

1. Import necessary R packages
2. Import arcPy library to leverage ArcGIS Pro functions
3. Wrangle data using `sf`
4. Write the data out to ArcGIS Pro for Analysis
5. Perform cluster analysis using ArcGIS Pro's `Density-Based Clustering`
6. Create interactive maps of clusters

### Step 1. Importing Necessary R libraries

In [1]:
library(arcgisbinding)
library(e1071)
library(leaflet)
library(leaflet.esri)
library(reticulate)
library(sf)

*** Please call arc.check_product() to define a desktop license.
Loading required package: leaflet.extras
Warning message:
"package 'leaflet.extras' was built under R version 3.5.3"Linking to GEOS 3.8.0, GDAL 3.0.4, PROJ 6.3.1


#### Initialize `arcgisbinding`
Next, we will run `arc.check_product()` to make enable `arcgisbinding`. This function will return:

1. The version of ArcGIS Pro
2. The type of ArcGIS Pro license
3. The version of the `arcgisbinding` package

In [2]:
arc.check_product()
reticulate::use_python('C:/Users/orhu8849/AppData/Local/ESRI/conda/envs/r_28_fix/python.exe', required = T)

product: ArcGIS Pro (12.8.0.28988)
license: Advanced
version: 1.0.1.244

### Step 2. Import arcPy Library
After a successful run of `arc.check_product` we will import `arcPy` library to call Geoprocessing functions from R.

We will set `arcPy`'s environment variable `overwriteOutput` to `True` to be able to allow overwriting output from Geoprocessing tools.

In [3]:
arcpy <- import("arcpy")
arcpy$env$overwriteOutput = TRUE
write.dir <- file.path(getwd(), 'Transshipment.gdb')

### Step 3. Wrangle data using `sf`

The original data lives in a `csv` file. We will use `sf` to convert this data into a Spatial R Dataframe. Once the dataframe is spatial, we will write it as a feature class in a file geodatabase.

In [4]:
csv.folder <- 'C:/Users/orhu8849/Documents/ArcGIS/Projects/Transshipment/csv'
## Read in Loitering Events
loitering.df <- read.csv(file = file.path(csv.folder, 'loitering-events-v20170717.csv'))
loitering.df['Event_Type'] = 'Loitering'
head(loitering.df)

transshipment_mmsi,starting_latitude,starting_longitude,ending_latitude,ending_longitude,starting_timestamp,ending_timestamp,median_speed_knots,total_event_duration,Event_Type
256064000,12.793800,-69.50323,12.727950,-69.73912,2017-09-06T02:11:32Z,2017-09-06T15:44:15Z,1.0910647,13.915556,Loitering
256064000,25.405013,-56.30280,25.393350,-56.48825,2017-07-22T13:19:49Z,2017-07-22T23:59:17Z,1.5483346,10.897639,Loitering
256064000,10.693413,-78.51280,10.484500,-78.88430,2017-11-29T14:48:51Z,2017-11-30T07:43:36Z,1.5034558,18.001389,Loitering
256064000,11.858933,-75.55338,12.007627,-75.53563,2017-09-07T23:39:03Z,2017-09-08T14:59:26Z,0.6503549,16.193611,Loitering
256064000,45.938747,-23.79696,45.976960,-23.44939,2017-08-17T23:18:21Z,2017-08-18T17:56:30Z,1.0098737,19.748333,Loitering
256064000,3.100107,-80.50907,3.170747,-80.43064,2017-09-11T03:02:21Z,2017-09-11T08:45:09Z,1.1113524,8.163333,Loitering


In [5]:
## Read in Encounter Events
encounter.df <- read.csv(file = file.path(csv.folder, 'encounter-events-v20170717.csv'))
encounter.df['Event_Type'] = 'Encounter'
head(encounter.df)

fishing_vessel_mmsi,transshipment_vessel_mmsi,start_time,end_time,mean_latitude,mean_longitude,duration_hr,median_distance_km,median_speed_knots,Event_Type
416565000,354240000,2016-11-18T14:30:00Z,2016-11-19T01:50:00Z,-17.03909,-79.06372,11.33333,0.03818823,0.58540192,Encounter
412679190,354240000,2016-12-11T14:50:00Z,2016-12-11T19:50:00Z,-20.26961,-79.24495,5.00000,0.02003331,0.57566267,Encounter
440863000,354240000,2017-06-13T12:50:00Z,2017-06-15T01:20:00Z,-62.64077,-60.69024,36.50000,0.05499231,0.01977473,Encounter
416563000,354240000,2016-11-15T11:30:00Z,2016-11-16T04:00:00Z,-17.04659,-79.06192,16.50000,0.03642717,1.02391706,Encounter
441309000,354240000,2017-05-19T00:40:00Z,2017-05-19T20:50:00Z,-46.62788,-60.55492,20.16667,0.03405300,0.54403111,Encounter
416562000,354240000,2016-11-17T13:30:00Z,2016-11-18T02:30:00Z,-17.22009,-79.07317,13.00000,0.04640635,0.46329246,Encounter


In [6]:
## Convert Tabular to Spatial Data Frame
encounter.sdf <- st_as_sf(encounter.df, coords = c("mean_longitude", "mean_latitude"))
st_crs(encounter.sdf) = 4326

## Convert Tabular to Spatial Data Frame
loitering.sdf <- st_as_sf(loitering.df, coords = c("starting_longitude", "starting_latitude"))
st_crs(loitering.sdf) = 4326

In [7]:
duration.skew <- skewness(encounter.sdf$duration_hr)
print(paste0("Transhipment duration distribution skewness:", duration.skew))

[1] "Transhipment duration distribution skewness:30.178759603994"


In [8]:
encounter.sdf['Duration_Log'] <- log1p(encounter.sdf$duration_hr) 

### Step 4. Write Out the Data to ArcGIS Pro for Analysis

In the next step, we rite out the spatial data to ArcGIS Pro for analysis and mapping.

In [9]:
encounter.fc <- file.path(write.dir, 'encounter')
loitering.fc <- file.path(write.dir, 'loitering')
arc.write(encounter.fc, encounter.sdf, overwrite = TRUE)
arc.write(loitering.fc, loitering.sdf, overwrite = TRUE)

### Step 5. Create Interactive Maps and Run `arcpy`

In the next step, we visualize transshipment events taking place globally. We use `esri.leaflet` to the operation below:

In [10]:
## Plot 
L<-leaflet(elementId='encounters') %>%
  addProviderTiles(providers$Esri.OceanBasemap) %>%
  addCircleMarkers(data = encounter.sdf,  
                   radius=5, 
                   label=~sprintf("Median Speed (Knots): %s", encounter.sdf$median_speed_knots))

<img src="map1.png"/>

### Step 6. Perform Density-Based Clustering of Transshipment Events

Next, we model clusters in transshipment events using `Density-Based Clustering` method in `arcPy`.

In [11]:
encounter.clus <- file.path(write.dir, "encounter_clusters")
arcpy$stats$DensityBasedClustering(encounter.fc, encounter.clus, "HDBSCAN", 500, NULL, NULL)

encounter.HDBScan <- arc.select(arc.open(encounter.clus), where_clause = 'CLUSTER_ID <> -1')
num.clust <- length(unique(encounter.HDBScan$COLOR_ID))

cluster.pal <- colorFactor(topo.colors(num.clust), domain=encounter.HDBScan$COLOR_ID)
L2<-leaflet(elementId='encounter_clusters') %>%
  addProviderTiles(providers$Esri.OceanBasemap) %>%
  addCircleMarkers(data = encounter.HDBScan,  
                   radius=5, 
                   color=~cluster.pal(encounter.HDBScan$COLOR_ID))

C:\Users\orhu8849\Documents\ArcGIS\Projects\Transshipment\Transshipment.gdb\encounter_clusters

<img src="map2.png"/>

In [ ]:
loitering.clus <- file.path(write.dir, "loitering_clusters")
arcpy$stats$DensityBasedClustering(loitering.fc, loitering.clus, "HDBSCAN", 500, NULL, NULL)


loitering.HDBScan <- arc.select(arc.open(loitering.clus), where_clause = 'CLUSTER_ID <> -1')
num.clust <- length(unique(loitering.HDBScan$COLOR_ID))

cluster.pal <- colorFactor(topo.colors(num.clust), domain=loitering.HDBScan$COLOR_ID)

L3 <-leaflet(elementId='encounter_clusters') %>%
  addProviderTiles(providers$Esri.OceanBasemap) %>%
  addCircleMarkers(data = loitering.HDBScan,  
                   radius=5, 
                   color=~cluster.pal(loitering.HDBScan$COLOR_ID))

<img src="map3.png"/>